In [170]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [171]:
datasets = 'train'
path = os.path.join(os.getcwd(), datasets , "*.xml")
all_xml = glob.glob(path)
len(all_xml)

3003

In [172]:
# 231.xml has error in parsing as it was not stored in VOC XML format
df = pd.DataFrame()
for i in all_xml:
    try:
        xml_df  = xml_to_csv(i)
        xml_df["image_name"] = i
        df = df.append(xml_df, ignore_index = True)
    except:
        continue

In [173]:
# EOL error if you put \ at the end of a string. Replacing it with \\ will work fine
left = "C:\\Users\\ASUS\\Downloads\\train\\" 
df['image_name'] = df['image_name'].map(lambda x: x.lstrip(left).rstrip('.xml')) + ".jpg"

In [174]:
df.drop("filename",axis=1, inplace = True)

In [180]:
df.head()

,width,height,class,xmin,ymin,xmax,ymax,image_name
0,1200,800,bus,833,390,1087,800,01.jpg
1,1200,800,bus,901,284,1018,395,01.jpg
2,1200,800,bus,909,241,1010,287,01.jpg
3,1200,800,rickshaw,761,413,832,540,01.jpg
4,1200,800,rickshaw,777,364,828,409,01.jpg


In [178]:
# checking
df["image_name"].nunique()

3002

In [181]:
# convert to csv 
df.to_csv("dhaka_ai.csv", index=False)